In [3]:
pip install scholarly


[notice] A new release of pip is available: 23.3.1 -> 24.1.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
import json
import shelve
import time
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor
from scholarly import scholarly

CACHE_FILE = 'scholarly_cache.db'

def get_author_info(author_name):
    search_query = scholarly.search_author(author_name)
    try:
        author = next(search_query)
        return scholarly.fill(author)
    except StopIteration:
        print(f"No author found for name: {author_name}")
        return None

def fill_publication(pub):
    with shelve.open(CACHE_FILE) as cache:
        if pub['author_pub_id'] in cache:
            return cache[pub['author_pub_id']]
        filled_pub = scholarly.fill(pub)
        cache[pub['author_pub_id']] = filled_pub
        return filled_pub

def extract_coauthors(author_info, years):
    current_year = datetime.now().year
    coauthors = set()

    def process_publication(pub):
        try:
            pub = fill_publication(pub)
            pub_year = pub['bib'].get('pub_year')
            if pub_year and isinstance(pub_year, int) and current_year - pub_year <= years:
                authors = pub['bib'].get('author', '')
                if authors:
                    coauthors.update(authors.split(' and '))
        except Exception as e:
            print(f"Error filling publication details: {e}")

    with ThreadPoolExecutor(max_workers=5) as executor:
        executor.map(process_publication, author_info['publications'])

    return coauthors

def main():
    author_name = input("Enter the author's name: ")
    years = int(input("Enter the number of years to look back for co-authorship: "))

    author_info = get_author_info(author_name)
    if author_info:
        print(f"Found author: {author_info['name']}")
        print(f"Total publications found: {len(author_info['publications'])}")

        coauthors = extract_coauthors(author_info, years)
        coauthors.discard(author_info['name'])
        print(f"Co-authors in the last {years} years: {coauthors}")
    else:
        print(f"Unable to retrieve information for author: {author_name}")

if __name__ == "__main__":
    main()


Found author: Dennis DeCoste
Total publications found: 89
Co-authors in the last 1 years: {'Sean Lie', 'Shreyas Saxena', 'William Marshall', 'Joel Hestness', 'Tianda Li', 'Vitaliy Chiley', 'Abhay Gupta', 'Vithursan Thangarasa', 'Kevin Leong', 'Anshul Samar'}
